In [ ]:
import os
import numpy as np


# return ksi, omegas, omega_d, freqs
def calFreqs(evals, beta, alpha):
    num_modes = len(evals)
    valid_map = np.zeros(num_modes)
    omegas = np.zeros(num_modes)
    omega_d = np.zeros(num_modes)
    ksi = np.zeros(num_modes)
    freqs = np.zeros(num_modes)

    for i in range(num_modes):
        if (evals[i] < 0):
            valid_map[i] = 0
            print('evals < 0 at ', i)
            continue

        omegas[i] = np.sqrt(evals[i])

        if (omegas[i] < 100 or omegas[i] > 2e5):
            print(f'omegas[{i}] = {omegas[i]} is out of 20hz 20000hz range')
            valid_map[i] = 0
            continue
        
        ksi[i] = (beta + alpha * evals[i]) / 2 / omegas[i]
        scale = 1 - ksi[i] * ksi[i]
        if (scale < 0 ):
            valid_map[i] = 0
            print('1 - ksi^2 < 0 at', i)
            continue

        omega_d[i] = omegas[i] * np.sqrt(scale)
        freqs[i] = 0.5 * omega_d[i] / np.pi
    return ksi, omegas, omega_d, freqs

# return mode_sample, samples
def genSound(ksi, omegas, omega_d, activation, fs, duration):
    num_modes = len(ksi)

    time_slot = np.arange(fs * duration) / fs

    mode_sample = np.zeros((num_modes, len(time_slot)))
    samples = np.zeros(len(time_slot))

    for i in range(num_modes):
        if(omega_d[i] != 0):
            amplitude = np.exp(time_slot * (-1) * ksi[i] * omegas[i]) * abs(activation[i]) / omega_d[i]
            mode_sample[i] = (np.sin(omega_d[i] * time_slot ) * amplitude).astype(np.float32)
            samples += mode_sample[i]
    return mode_sample, samples

def saveSound(filename, fs, samples):
    from scipy.io.wavfile import write
    tmp_samples = samples * 1e6
    write(filename, fs, tmp_samples)

def freq2mel(freq):
    return 2596.0 * np.log10(1 + freq / 700.0)

def eval2freq(evals):
    return np.sqrt(evals) * 0.5 / np.pi

def eval2mel(evals):
    return freq2mel(eval2freq(evals))

In [ ]:
filename = 'T'+'598'

In [ ]:
GROUND = False
PRED = True
RESNET = False

gt_dir = '../DATA/eigen'
# gt_dir = './EXP/1/glass'
# gt_dir = './EXP/tmp/modelnet40'
if(PRED):
    pred_dir = '../DATA/test_results'
    # pred_dir = './EXP/tmp/pred'
    if(RESNET):
        resnet_dir = '../DATA/resnet_test_results'

# obj_file = os.path.join(obj_dir, filename+'.obj')
gt_file = os.path.join(gt_dir, filename, 'eigen.npz')
# gt_file = './EXP/1/glass353/eigen.npz'

if(PRED):
    pred_file = os.path.join(pred_dir, filename+'.npy')
    if(RESNET):
        resnet_file = os.path.join(resnet_dir, filename+'.npy')

gt_read = np.load(gt_file)
evecs = gt_read['evecs']
gt = gt_read['evals']

if(PRED):
    pred = np.load(pred_file)
    pred = pred * 1e9
    if(RESNET):
        res_pred = np.load(resnet_file)
        res_pred = res_pred * 1e9

###### Cal Freqs #####
gt_ksi, gt_omegas, gt_omega_d, gt_freqs = calFreqs(gt, beta=5.0, alpha=1e-7)
if(PRED):
    pd_ksi, pd_omegas, pd_omega_d, pd_freqs = calFreqs(pred, beta=5.0, alpha=1e-7)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import MultipleLocator
plt.style.use('seaborn')
# print(plt.style.available) # 打印样式列表

##### CONFIG ######
DOT_SIZE = 60
COLOR = 'steelblue'
MARKER = 'o'
ALPHA = 0.8
LINE_WIDTH = 0.3
EDGE_COLOR = 'white'

# x = np.arange(50)

plt.figure(figsize=(6,6))
plt.scatter(
            gt_freqs[3:50],
            pd_freqs[3:50],
            # freq2mel(gt_freqs),
            # freq2mel(pd_freqs),
            s = DOT_SIZE,
            c = COLOR,
            marker = MARKER,
            alpha = ALPHA,
            linewidths = LINE_WIDTH,
            edgecolors = EDGE_COLOR
            )

plt.xlabel('Ground Truth Freq')
plt.ylabel('ModalNet Predition')
plt.title('ModalNet')

# BEGIN = None
# END = None
BEGIN = 1000
END = 8000
plt.xlim(BEGIN, END)
plt.ylim(BEGIN, END)

ax=plt.gca()
ax.set_aspect(1)
plt.show()

### Forces

In [ ]:
###### Set Force #####
contact_pos = 190
contact_force = [0, 0, -1]
activation = np.zeros(50)
for dir in range(3):
    activation += contact_force[dir] * evecs[3*contact_pos + dir]
# activation

### Post & Save

In [ ]:
Ceramic =   2700,   7.4E10, 0.19,   5,  1E-7
Glass =     2600,   6.2E10, 0.20,   1,  1E-7
Wood =      750,    1.1E10, 0.25,   60, 2E-6
Plastic =   1070,   1.4E9,  0.35,   30, 1E-6
Iron =      8000,   2.1E11, 0.28,   5,  1E-7
Polycarb =  1190,   2.4E9,  0.37,   0.5,4E-7
Steel =     7850,   2.0E11, 0.29,   5,  3E-8
Tin =       7265,   5e10,   0.325,  2,  3E-8

def omega_rate(src, target, rescale):
    k1 = target[1] / src[1]
    k2 = target[0] / src[0]
    return k1**0.5*k2**(-0.5)/rescale

def activation_rate(src, target, rescale):
    k1 = target[1] / src[1]
    k2 = target[0] / src[0]
    return k2**(-0.5)*rescale**(-3/2)

In [ ]:
rescale = 0.75
rate_freq = omega_rate(Glass, Glass, rescale)
rate_act = activation_rate(Glass, Glass, rescale)
print(rate_freq, rate_act)

In [ ]:
###### Save as file #####
duration = 2
fs = 44100
if(GROUND):
    gt_mode_sample, gt_sample = genSound(gt_ksi, gt_omegas, gt_omega_d * rate_freq, activation * rate_act, fs, duration)
if(PRED):
    pd_mode_sample, pd_sample = genSound(pd_ksi, gt_omegas, gt_omega_d * rate_freq, activation * rate_act, fs, duration)
    # pd_mode_sample, pd_sample = genSound(pd_ksi, pd_omegas, pd_omega_d * rate_freq, activation * rate_act, fs, duration)

if(GROUND):
    saveSound(os.path.join('./EXP/1/glass353', filename+'-p'+str(contact_pos)+'-s'+str(rescale) +'_gt.wav'), fs, gt_sample)
if(PRED):
    saveSound(os.path.join('./EXP/1/glass353', filename+'-p'+str(contact_pos)+'-s'+str(rescale) +'_test.wav'), fs, pd_sample)